In [ ]:
import datasets
from datasets import Dataset
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-1.5B-v3.0-Instruct")
model = AutoModelForCausalLM.from_pretrained("speakleash/Bielik-1.5B-v3.0-Instruct").to(DEVICE)

In [7]:
dataset = datasets.load_dataset("Igorrr0/polish-qa-general")


def to_chat_format(sample):
    system_prompt = {"role": "system", "content": "Odpowiadaj krótko i konwersacyjnie :)"}
    user_msg = {"role": "user", "content": sample["instruction"]}
    assistant_msg = {"role": "assistant", "content": sample["output"]}
    return {"prompt": [system_prompt, user_msg], "completion": [system_prompt, user_msg, assistant_msg]}

dataset = dataset.map(to_chat_format).remove_columns(["instruction", "input", "output"])

Dataset({
    features: ['prompt'],
    num_rows: 368
})

In [40]:
import transformers
from transformers.pipelines.pt_utils import KeyDataset

pipe = TextGenerationPipeline(
    model=model, 
    tokenizer=tokenizer,
    batch_size=1,
    device=DEVICE,
)

gc.collect()

train_dataset = KeyDataset(dataset["train"], "prompt")

print("Generating responses")
generated = pipe(train_dataset, batch_size=64, max_new_tokens=20, do_sample=True)
print("Done")

Device set to use cuda


Generating responses


In [51]:
print(generated[0])
list_dataset = [{"completion": x, "prompt": dataset["train"][i]["prompt"]} for i, x in enumerate(generated)]
idx = int(len(list_dataset) * 0.8)  # 80/20 split
cached_train = Dataset.from_list(list_dataset[:idx])
cached_train.to_json("train.json")
cached_test = Dataset.from_list(list_dataset[idx:len(list_dataset)])
cached_test.to_json("test.json")

[{'generated_text': [{'content': 'Odpowiadaj krótko i konwersacyjnie :)', 'role': 'system'}, {'content': 'Jak mogę pomóc?', 'role': 'user'}, {'role': 'assistant', 'content': 'Jestem tu, aby pomóc w każdej sprawie, która przychodzi Ci do głowy, od prostych pytań po bardziej'}]}]


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

6303